## About this notebook

*[Jigsaw Multilingual Toxic Comment Classification](https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification)* is the 3rd annual competition organized by the Jigsaw team. It follows *[Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)*, the original 2018 competition, and *[Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification)*, which required the competitors to consider biased ML predictions in their new models. This year, the goal is to use english only training data to run toxicity predictions on many different languages, which can be done using multilingual models, and speed up using TPUs.

Many awesome notebooks has already been made so far. Many of them used really cool technologies like [Pytorch XLA](https://www.kaggle.com/theoviel/bert-pytorch-huggingface-starter). This notebook instead aims at constructing a **fast, concise, reusable, and beginner-friendly model scaffold**. It will focus on the following points:
* **Using Tensorflow and Keras**: Tensorflow is a powerful framework, and Keras makes the training process extremely easy to understand. This is especially good for beginners to learn how to use TPUs, and for experts to focus on the modelling aspect.
* **Using Huggingface's `transformers` library**: [This library](https://huggingface.co/transformers/) is extremely popular, so using this let you easily integrate the end result into your ML pipelines, and can be easily reused for your other projects.
* **Native TPU usage**: The TPU usage is abstracted using the native `strategy` that was created using Tensorflow's `tf.distribute.experimental.TPUStrategy`. This avoids getting too much into the lower-level aspect of TPU management.
* **Use a subset of the data**: Instead of using the entire dataset, we will only use the 2018 subset of the data available, which makes this much faster, all while achieving a respectable accuracy.

In [ ]:
import os
import gc
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
import traitlets
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from sklearn.metrics import roc_auc_score

warnings.simplefilter("ignore")

np.random.seed(100)

## Helper Functions

In [ ]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [ ]:
def loss_fn(y_true, y_pred):
    loss = tf.keras.losses.binary_crossentropy(y_true, y_pred,
         label_smoothing=LABEL_SMOOTHING)
    loss = tf.reduce_mean(loss)
    return loss

In [ ]:
def build_model(transformer, loss='binary_crossentropy', max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    emb = transformer(input_word_ids)[0]
    _avg = tf.keras.layers.GlobalAveragePooling1D()(emb)
    _max = tf.keras.layers.GlobalMaxPooling1D()(emb)
    x = tf.keras.layers.Concatenate()([_avg, _max])
    x = tf.keras.layers.Dropout(0.15)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=3e-5), loss=loss_fn , metrics=[tf.keras.metrics.AUC()])
    
    return model

## TPU Configs

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Data access
#GCS_DS_PATH = KaggleDatasets().get_gcs_path('kaggle/input/') 

## Create fast tokenizer

# First load the real tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

## Load text data into memory

In [ ]:
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [ ]:
lang_densety = test.groupby('lang').count()['id']/ len(test)
target_densety = valid.groupby('toxic').count()['id']/ len(valid)

In [ ]:
lang_densety

In [ ]:
target_densety

In [ ]:
N_SAMPLES = 500000

train_dfs = []
for lang in ['es', 'it', 'pt', 'tr', 'ru', 'fr']:
    _df = pd.read_csv(f"/kaggle/input/jigsaw-train-multilingual-coments-google-api/jigsaw-toxic-comment-train-google-{lang}-cleaned.csv")
    
    _df0 = _df.loc[_df.toxic == 0, :]
    _df1 = _df.loc[_df.toxic == 1, :]
    
    n_samples = int(N_SAMPLES * lang_densety[lang])
    
    n_samples_0 = int(n_samples * target_densety[0])
    n_samples_1 = int(n_samples * target_densety[1])
    
    _df0 = _df0.sample(n_samples_0)
    _df1 = _df1.sample(n_samples_1)
    
    train_dfs.append(pd.concat([_df0, _df1], ignore_index=True).sample(n_samples_0 + n_samples_1))
    
train = pd.concat(train_dfs, ignore_index=True)
train = train.sample(min(N_SAMPLES, len(train)))
train.toxic = train.toxic.round().astype(int)

In [ ]:
train.groupby('toxic').count()['id']/ len(train)

In [ ]:
class TextTransformation:
    def __call__(self, text: str, lang: str = None) -> tuple:
        raise NotImplementedError('Abstarct')
        
class LowerCaseTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        return text.lower(), lang
    
class PunctuationTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for p in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’' +"/-'" + "&" + "¡¿":
            if '’' in text:
                text = text.replace('’', f' \' ')
                
            if '’' in text:
                text = text.replace('’', f' \' ')
              
            if '—' in text:
                text = text.replace('—', f' - ')
                
            if '–' in text:
                text = text.replace('–', f' - ')   
              
            if '“' in text:
                text = text.replace('“', f' " ')   
                
            if '«' in text:
                text = text.replace('«', f' " ')   
                
            if '»' in text:
                text = text.replace('»', f' " ')   
            
            if '”' in text:
                text = text.replace('”', f' " ') 
                
            if '`' in text:
                text = text.replace('`', f' \' ')              

            text = text.replace(p, f' {p} ')
                
        return text.strip(), lang
    
class NumericTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in range(10):
            text = text.replace(str(i), f' {str(i)} ')
        return text, lang
    
class ETransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['\u00E8', '\u00E9', '\u00EA', '\u00EB', '\u0450', '\u0451']:
            text = text.replace(i, 'e')
        return text, lang
    
class ATransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['à', 'á', '\u00E2', '\u00E3', '\u00E4', '\u00E5']:
            text = text.replace(i, 'a')
        return text, lang
    
class OTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['ó', 'ò', 'ö','õ', 'ô']:
            text = text.replace(i, 'o')
        return text, lang
    

class CTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['ç']:
            text = text.replace(i, 'c')
        return text, lang
    
class ITransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['í', 'ı', 'ì']:
            text = text.replace(i, 'i')
        return text, lang
    
class STransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['ş']:
            text = text.replace(i, 's')
        return text, lang
    
    
class NTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['ñ', 'n']:
            text = text.replace(i, 'n')
        return text, lang
    
    
class UTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['ù', 'ü', 'û', 'ú']:
            text = text.replace(i, 'u')
        return text, lang
    
class GTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['ğ']:
            text = text.replace(i, 'g')
        return text, lang

class RTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in ['r']:
            text = text.replace(i, 'r')
        return text, lang
    
class WikiTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        text = text.replace('wikiproject', ' wiki project ')
        for i in [' vikipedi ', ' wiki ', ' википедии ', " вики ", ' википедия ', ' viki ', ' wikipedien ', ' википедию ']:
            text = text.replace(i, ' wikipedia ')
        return text, lang
    
class PixelTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        for i in [' px ']:
            text = text.replace(i, ' pixel ')
        return text, lang
    
    
class RuTransformation(TextTransformation):
    def __call__(self, text: str, lang: str = None) -> tuple:
        if lang is not None and lang == 'ru' and 'http' not in text and 'jpg' not in text and 'wikipedia' not in text:
            text = text.replace('t', 'т')
            text = text.replace('h', 'н')
            text = text.replace('b', 'в')
            text = text.replace('c', 'c')
            text = text.replace('k', 'к')
            text = text.replace('e', 'е')
            text = text.replace('a', 'а')
        return text, lang
    
    
class CombineTransformation(TextTransformation):
    def __init__(self, transformations: list):
        self._transformations = transformations
        
    def __call__(self, text: str, lang: str = None) -> tuple:
        for transformation in self._transformations:
            text, lang = transformation(text, lang)
        return text, lang
    
    def append(self, transformation: TextTransformation):
        self._transformations.append(transformation)
        
        
transformer = CombineTransformation(
    [
        LowerCaseTransformation(),
        PunctuationTransformation(),
        NumericTransformation(),
        ETransformation(),
        ATransformation(),
        OTransformation(),
        CTransformation(),
        ITransformation(),
        STransformation(),
        UTransformation(),
        GTransformation(),
        NTransformation(),
        WikiTransformation(),
        PixelTransformation(),
        RuTransformation()
    ]
)

In [ ]:
train['comment_text'] = [v[0] for v in train.apply(lambda x: transformer(x.comment_text), axis=1).values]
valid['comment_text'] = [v[0] for v in valid.apply(lambda x: transformer(x.comment_text, x.lang), axis=1).values]
test['content'] = [v[0] for v in test.apply(lambda x: transformer(x.content, x.lang), axis=1).values]

In [ ]:
fast_tokenizer = BertWordPieceTokenizer('../input/bert-tokenizer-vocabulary-analiz/new_bert_base_multilingual_uncased/vocab.txt', lowercase=True)
fast_tokenizer

## Fast encode

In [ ]:
x_train = fast_encode(train.comment_text.astype(str), fast_tokenizer, maxlen=512)
x_valid = fast_encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=512)
x_test = fast_encode(test.content.astype(str), fast_tokenizer, maxlen=512)
y_train = train.toxic.values
y_valid = valid.toxic.values

del train_dfs, train, valid, test
gc.collect()

## Build datasets objects

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(64)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(64)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(64)
)


# Focal Loss

In [ ]:
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=.15):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

## Load model into the TPU

In [ ]:
model = transformers.TFBertModel.from_pretrained('bert-base-multilingual-uncased')
model.resize_token_embeddings(fast_tokenizer.get_vocab_size())

save_path = '/kaggle/working/new_bert_base_multilingual_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
model.save_pretrained(save_path)

In [ ]:
os.listdir(save_path)

In [ ]:
%%time
with strategy.scope():
    transformer_layer = transformers.TFBertModel.from_pretrained(save_path, from_pt=False)
    model = build_model(transformer_layer, loss=focal_loss(gamma=1.5), max_len=512)
model.summary()

## RocAuc Callback

In [ ]:
from tensorflow.keras.callbacks import Callback 

class RocAucCallback(Callback):
    def __init__(self, test_data, score_thr):
        self.test_data = test_data
        self.score_thr = score_thr
        self.test_pred = []
        
    def on_epoch_end(self, epoch, logs=None):
        if logs['val_auc'] > self.score_thr:
            print('\nRun TTA...')
            self.test_pred.append(self.model.predict(self.test_data))

# LrScheduler

In [ ]:
def build_lrfn(lr_start=0.000001, lr_max=0.000004, 
               lr_min=0.0000001, lr_rampup_epochs=5, 
               lr_sustain_epochs=3, lr_exp_decay=.87):
    lr_max = lr_max * strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    
    return lrfn

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))

_lrfn = build_lrfn()
plt.plot([i for i in range(35)], [_lrfn(i) for i in range(35)]);

## Train Model

### First Stage

In [ ]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
er = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=20, restore_best_weights=True, mode='max')

train_history = model.fit(
    train_dataset,
    steps_per_epoch=200,
    validation_data=valid_dataset,
    callbacks=[lr_schedule, er],
    epochs=35
)

In [ ]:
bert_weights = model.get_weights()

### Second Stage

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))

lrfn = build_lrfn(lr_start=0.000001, lr_max=0.000001, 
               lr_min=0.0000001, lr_rampup_epochs=2, 
               lr_sustain_epochs=1, lr_exp_decay=.65)
plt.plot([i for i in range(15)], [lrfn(i) for i in range(15)]);

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [ ]:
skf = StratifiedKFold(n_splits=5)

test_preds = np.zeros((len(x_test),))

for tr_idx, vl_idx in skf.split(x_valid, y_valid):
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
    er = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=20, restore_best_weights=True, mode='max')
    x_tr, x_val, y_tr, y_val = x_valid[tr_idx], x_valid[vl_idx], y_valid[tr_idx], y_valid[vl_idx]

    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_tr, y_tr))
        .repeat()
        .shuffle(2048)
        .batch(32)
        .prefetch(AUTO)
    )

    valid_dataset = (
        tf.data.Dataset
        .from_tensor_slices((x_val, y_val))
        .batch(32)
        .cache()
        .prefetch(AUTO)
    )
    
    model.set_weights(bert_weights)
    train_history = model.fit(
        train_dataset,
        steps_per_epoch=40,
        validation_data=valid_dataset,
        callbacks=[lr_schedule, er],
        epochs=15)
    test_preds += model.predict(test_dataset)[:, 0] / 5

## Submission

In [ ]:
sub['toxic'] = test_preds
sub.to_csv('submission.csv', index=False)

# Reference
* [Jigsaw TPU: DistilBERT with Huggingface and Keras](https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras)
* [inference of bert tpu model ml w/ validation](https://www.kaggle.com/abhishek/inference-of-bert-tpu-model-ml-w-validation)
* [Overview of Text Similarity Metrics in Python](https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50)
* [test-en-df](https://www.kaggle.com/bamps53/test-en-df)
* [val_en_df](https://www.kaggle.com/bamps53/val-en-df)
* [Jigsaw multilingual toxic - test translated](https://www.kaggle.com/kashnitsky/jigsaw-multilingual-toxic-test-translated)